In [1]:
import requests
from scrapy.selector import Selector
import re
from requests_html import HTMLSession
import json
import time
from tqdm import tqdm

In [139]:
r = requests.get('https://ejournal.upsi.edu.my/index.php/MJM/issue/archive')
session = HTMLSession()

In [164]:
pattern = re.compile('[\\t\\n]+')
main_data = []
items = Selector(r).css('h2')
for x in items:
    
    # Main page
    archive_title = "Malaysian Journal of Music " + re.sub(pattern, '', x.css('div::text').get())
    archive_url = x.css('a::attr(href)').get()

    print(archive_title,'\n')
    # Parse archive page
    archive_page = requests.get(archive_url)
    article_abstract_urls = Selector(archive_page).css('div.section ul h3 a::attr(href)').getall()

    # Parse article abstracts
    for article_url in article_abstract_urls:
        temp_data = {}
        article_extract_page = session.get(article_url, timeout=10, )
        temp_data['archive_title'] = archive_title
        temp_data['archive_url'] = archive_url
        temp_data['article_url'] = article_url
        temp_data['article_title'] = Selector(article_extract_page).css('h1.page_title::text').get()
        # print(article_url, ' @ ', temp_data['article_title'],'\n')
        temp_data['article_keywords'] = Selector(article_extract_page).css('div.main_entry section.keywords > span.value::text').get()
        temp_data['article_abstract_text'] = Selector(article_extract_page).css('div.main_entry section.item > p::text').get()
        temp_data['article_reference_list'] = Selector(article_extract_page).css('div.main_entry section.item div.value p::text').getall()
        temp_data['article_pdf_url'] = Selector(article_extract_page).css('div.entry_details > div > ul a::attr(href)').get()
        # time.sleep(3)

        main_data.append(temp_data)

Malaysian Journal of Music Vol. 12 No. 2 (2023) 

https://ejournal.upsi.edu.my/index.php/MJM/article/view/8101  @  
		Problems and Expectations of Students Regarding the Management of Learning in the Subjects of Thai Music Theory and History  within the Thai Undergraduate Music Education Curriculum
	 

https://ejournal.upsi.edu.my/index.php/MJM/article/view/8537  @  
		Kidung:
	 

https://ejournal.upsi.edu.my/index.php/MJM/article/view/8108  @  
		Towards Eco-Organology and Paper Flute Design to Disrupt the Plastic Recorder Industry
	 

https://ejournal.upsi.edu.my/index.php/MJM/article/view/8079  @  
		Calung Banyumasan:
	 

https://ejournal.upsi.edu.my/index.php/MJM/article/view/8098  @  
		Gender Differences in the Function of Music for Emotion Regulation Development in Everyday Life:  
	 

https://ejournal.upsi.edu.my/index.php/MJM/article/view/8081  @  
		 Investigating the Roles of Pedagogical Content Knowledge in Music Education: 
	 

https://ejournal.upsi.edu.my/index.php/MJM/a

In [165]:
import pandas as pd
df = pd.DataFrame(main_data)
for col in df.columns :
    if col != 'article_reference_list':
        df[col] = df[col].str.replace('[\\t\\n]', '', regex=True)

In [167]:
df.to_json('journals_with_abstract_only.json', 'records')